In [ ]:
# heatmap latitud longitud

import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "folium"])
import folium, sys
print("Folium:", folium.__version__, "Python:", sys.executable)
from folium.plugins import HeatMap


puntos = []
for u in usuarios:
    try:
        lat = float(u["location"]["coordinates"]["latitude"])
        lon = float(u["location"]["coordinates"]["longitude"])
        if -90 <= lat <= 90 and -180 <= lon <= 180:
            puntos.append([lat, lon])
    except (KeyError, ValueError, TypeError):
        pass

# Centro del mapa
if puntos:
    lat_c = sum(p[0] for p in puntos) / len(puntos)
    lon_c = sum(p[1] for p in puntos) / len(puntos)
    
else:
    lat_c, lon_c = 20, 0

m = folium.Map(location=[lat_c, lon_c], zoom_start=2, tiles="OpenStreetMap")
HeatMap(puntos, radius=12, blur=10, min_opacity=0.3).add_to(m)
m.save("heatmap_usuarios.html")
print("Mapa guardado en heatmap_usuarios.html")

In [ ]:
#mapa paises
%pip install pycountry

from collections import Counter
import pandas as pd, requests, folium, os, webbrowser, pathlib, sys, subprocess

# 1) Conteo por país desde 'usuarios'
conteo = Counter(u["location"]["country"] for u in usuarios)
df = pd.DataFrame(conteo.items(), columns=["country", "count"])

url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"
world = requests.get(url).json()


use_iso3 = True
try:
    import pycountry
except ModuleNotFoundError:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pycountry"])
        import pycountry
    except Exception:
        print("pycountry no disponible: usaré coincidencia por nombre.")
        use_iso3 = False

def to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except Exception:
        return None


name_map = {
    "United States": "United States of America",
    "Russia": "Russian Federation",
    "Iran": "Iran (Islamic Republic of)",
    "Syria": "Syrian Arab Republic",
    "Moldova": "Republic of Moldova",
    "Tanzania": "United Republic of Tanzania",
    "Vietnam": "Viet Nam",
    "Laos": "Lao People's Democratic Republic",
    "South Korea": "Korea, Republic of",
    "North Korea": "Korea, Democratic People's Republic of",
    "Cape Verde": "Cabo Verde",
    "Ivory Coast": "Côte d'Ivoire",
    "Czechia": "Czech Republic",
    "Swaziland": "Eswatini",
    "The Bahamas": "Bahamas",
    "The Gambia": "Gambia",
    "Burma": "Myanmar",
    "North Macedonia": "Macedonia",  
    "Venezuela": "Venezuela (Bolivarian Republic of)",
    "Bolivia": "Bolivia (Plurinational State of)",
    "Micronesia": "Micronesia (Federated States of)",
    "Brunei": "Brunei Darussalam",
}


m = folium.Map(location=[20, 0], zoom_start=2, tiles="OpenStreetMap")

if use_iso3:
    df_iso = df.copy()
    df_iso["iso3"] = df_iso["country"].apply(to_iso3)
    df_iso = df_iso.dropna(subset=["iso3"])
    folium.Choropleth(
        geo_data=world,
        data=df_iso,
        columns=["iso3", "count"],
        key_on="feature.id",          
        fill_color="YlOrRd",
        nan_fill_color="#eeeeee",
        legend_name="Usuarios por país",
    ).add_to(m)
else:
    df_nm = df.copy()
    df_nm["world_name"] = df_nm["country"].apply(lambda x: name_map.get(x, x))
    folium.Choropleth(
        geo_data=world,
        data=df_nm,
        columns=["world_name", "count"],
        key_on="feature.properties.name",  
        fill_color="YlOrRd",
        nan_fill_color="#eeeeee",
        legend_name="Usuarios por país",
    ).add_to(m)


out = "choropleth_usuarios.html"
m.save(out)
ruta = os.path.abspath(out)
print("Mapa por países:", ruta)
webbrowser.open(pathlib.Path(ruta).as_uri())

